In [45]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np 
from lifelines import KaplanMeierFitter
import os 
from matplotlib_venn import venn3
import pyper as pr
os.chdir('/data/master/DS175/share/0_Database')


In [65]:
conn = sqlite3.connect('Gliocova.db')
cur = conn.cursor()
HESAPC_oligo = pd.read_sql("SELECT * FROM HESAPC WHERE PSEUDO_PATIENTID IN (SELECT PSEUDO_PATIENTID FROM TUM_PAT WHERE MORPH_ICD10_O2 IN (9450, 9451) AND (INTERVAL_DIAG_TO_VS != '') AND (VITALSTATUS IN ('A', 'D')) AND (ETHNICITY IN ('A','B', 'C', 'D', 'F', 'G', 'H', 'J', 'L', 'M', 'N', 'P', 'R', 'S', 'X', 'Z')) AND (DIAGNOSISYEAR IN (2013,2014,2015,2016,2017,2018)) AND (SEX IN (1,2)) AND (BASISOFDIAGNOSIS IN (1,2,7,9)) AND (BIGTUMOURCOUNT IN (1,2,3,4)));",conn)

HESAPC_oligo.drop_duplicates(subset='PSEUDO_PATIENTID', inplace=True)
print(len(HESAPC_oligo))
bin = [-10000, -90, -60, -30, 0, 30, 60, 90, 180, 365, 1095, 1825, 10000]
label = ['less_than_-3months', 'less_than_-2months', 'less_than_-1month', 'more_than_-1month', 'less_than_1month', 'more_than_1month', 'more_than_2months', 'more_than_3months', 'more_than_0.5yrs', 'more_than_1yr', 'more_than_3yrs', 'more_than_5yrs']
DA_group = pd.cut(HESAPC_oligo['INTERVAL_DIAG_ADMISSION'].astype(int), bins = bin, labels = label)

DA_group.value_counts(sort=False).to_frame()


1326


,INTERVAL_DIAG_ADMISSION
less_than_-3months,0
less_than_-2months,21
less_than_-1month,33
more_than_-1month,240
less_than_1month,29
more_than_1month,19
more_than_2months,18
more_than_3months,77
more_than_0.5yrs,207
more_than_1yr,363


In [4]:
HESAPC_oligo['INTERVAL_DIAG_DECISION_ADMIT'].replace(r'', np.nan, inplace = True)
HESAPC_oligo.dropna(axis = 0, subset=['INTERVAL_DIAG_DECISION_ADMIT'], inplace = True)
bin = [-10000, -90, -60, -30, 0, 30, 60, 90, 180, 365, 1095, 1825, 10000]
label = ['less_than_-3months', 'less_than_-2months', 'less_than_-1month', 'more_than_-1month', 'less_than_1month', 'more_than_1month', 'more_than_2months', 'more_than_3months', 'more_than_0.5yrs', 'more_than_1yr', 'more_than_3yrs', 'more_than_5yrs']
DDA_group = pd.cut(HESAPC_oligo['INTERVAL_DIAG_DECISION_ADMIT'].astype(float).astype(int), bins = bin, labels = label)

DDA_group.value_counts(sort=False).to_frame()


,INTERVAL_DIAG_DECISION_ADMIT
less_than_-3months,9
less_than_-2months,9
less_than_-1month,29
more_than_-1month,104
less_than_1month,7
more_than_1month,9
more_than_2months,11
more_than_3months,76
more_than_0.5yrs,95
more_than_1yr,158


In [6]:
HESAPC_oligo.groupby(['ADMIMETH'])['PSEUDO_PATIENTID'].count()

ADMIMETH
11    283
12     87
13    260
Name: PSEUDO_PATIENTID, dtype: int64

In [7]:
HESAPC_oligo.groupby(['ADMISORC'])['PSEUDO_PATIENTID'].count()

ADMISORC
19    619
29      2
51      8
65      1
Name: PSEUDO_PATIENTID, dtype: int64

In [9]:
HESAPC_oligo['ELECDUR'].replace(r'', np.nan, inplace = True)
HESAPC_oligo.dropna(axis = 0, subset=['ELECDUR'], inplace = True)
bin = [0, 30, 60, 90, 180, 365, 1095, 1825, 10000]
label = ['less_than_1month', 'more_than_1month', 'more_than_2months', 'more_than_3months', 'more_than_0.5yrs', 'more_than_1yr', 'more_than_3yrs', 'more_than_5yrs']
Waiting_time_group = pd.cut(HESAPC_oligo['ELECDUR'].astype(float).astype(int), bins = bin, labels = label)

Waiting_time_group.value_counts(sort=False).to_frame()

,ELECDUR
less_than_1month,316
more_than_1month,131
more_than_2months,48
more_than_3months,43
more_than_0.5yrs,9
more_than_1yr,2
more_than_3yrs,0
more_than_5yrs,0


In [10]:
HESAPC_oligo.groupby(['CLASSPAT'])['PSEUDO_PATIENTID'].count()

CLASSPAT
1    240
2    251
3     58
Name: PSEUDO_PATIENTID, dtype: int64

In [15]:
HESAPC_oligo['INTERVAL_ADMISION_DISCHARGE'].replace(r'', np.nan, inplace = True)
HESAPC_oligo.dropna(axis = 0, subset=['INTERVAL_ADMISION_DISCHARGE'], inplace = True)
bin = [0, 30, 60, 90, 180, 365, 1095, 1825, 10000]
# label = ['less_than_1month', 'more_than_1month', 'more_than_2months', 'more_than_3months', 'more_than_0.5yrs', 'more_than_1yr', 'more_than_3yrs', 'more_than_5yrs']
Waiting_time_group = pd.cut(HESAPC_oligo['INTERVAL_ADMISION_DISCHARGE'].astype(int), bins = bin)

Waiting_time_group.value_counts(sort=False).to_frame()

,INTERVAL_ADMISION_DISCHARGE
"(0, 30]",213
"(30, 60]",4
"(60, 90]",2
"(90, 180]",0
"(180, 365]",0
"(365, 1095]",0
"(1095, 1825]",0
"(1825, 10000]",0


In [16]:
HESAPC_oligo.groupby(['DISMETH'])['PSEUDO_PATIENTID'].count()

DISMETH
1    534
2      3
4      1
8      2
Name: PSEUDO_PATIENTID, dtype: int64

In [18]:
HESAPC_oligo.groupby(['DISDEST'])['PSEUDO_PATIENTID'].count()

DISDEST
19    528
29      1
51      6
79      1
85      2
88      1
98      1
Name: PSEUDO_PATIENTID, dtype: int64

In [19]:
HESAPC_oligo['BEDYEAR'].replace(r'', np.nan, inplace = True)
HESAPC_oligo.dropna(axis = 0, subset=['BEDYEAR'], inplace = True)
bin = [0, 30, 60, 90, 180, 365, 1095, 1825, 10000]
# label = ['less_than_1month', 'more_than_1month', 'more_than_2months', 'more_than_3months', 'more_than_0.5yrs', 'more_than_1yr', 'more_than_3yrs', 'more_than_5yrs']
Waiting_time_group = pd.cut(HESAPC_oligo['BEDYEAR'].astype(int), bins = bin)

Waiting_time_group.value_counts(sort=False).to_frame()

,BEDYEAR
"(0, 30]",215
"(30, 60]",3
"(60, 90]",1
"(90, 180]",0
"(180, 365]",0
"(365, 1095]",0
"(1095, 1825]",0
"(1825, 10000]",0


In [42]:
HESAPC_oligo.groupby('STHATRET')['PSEUDO_PATIENTID'].count()

STHATRET
Q30    2
Q31    1
Q32    1
Q33    1
Q34    2
Q35    1
Q36    1
Name: PSEUDO_PATIENTID, dtype: int64

In [52]:
###################################### FOR IN PATIENTS ###########################################
HESOP_oligo = pd.read_sql("SELECT * FROM HESOP WHERE PSEUDO_PATIENTID IN (SELECT PSEUDO_PATIENTID FROM TUM_PAT WHERE MORPH_ICD10_O2 IN (9450, 9451) AND (INTERVAL_DIAG_TO_VS != '') AND (VITALSTATUS IN ('A', 'D')) AND (ETHNICITY IN ('A','B', 'C', 'D', 'F', 'G', 'H', 'J', 'L', 'M', 'N', 'P', 'R', 'S', 'X', 'Z')) AND (DIAGNOSISYEAR IN (2013,2014,2015,2016,2017,2018)) AND (SEX IN (1,2)) AND (BASISOFDIAGNOSIS IN (1,2,7,9)) AND (BIGTUMOURCOUNT IN (1,2,3,4)));",conn)

HESOP_oligo.drop_duplicates(subset='PSEUDO_PATIENTID', inplace=True)
print(len(HESOP_oligo))
HESOP_oligo['INTERVAL_DIAG_APPT'].replace(r'', np.nan, inplace = True)
HESOP_oligo.dropna(axis = 0, subset=['INTERVAL_DIAG_APPT'], inplace = True)
bin = [-10000, -90, -60, -30, 0, 30, 60, 90, 180, 365, 1095, 1825, 10000]
label = ['less_than_-3months', 'less_than_-2months', 'less_than_-1month', 'more_than_-1month', 'less_than_1month', 'more_than_1month', 'more_than_2months', 'more_than_3months', 'more_than_0.5yrs', 'more_than_1yr', 'more_than_3yrs', 'more_than_5yrs']
DDA_group = pd.cut(HESOP_oligo['INTERVAL_DIAG_APPT'].astype(float).astype(int), bins = bin, labels = label)

DDA_group.value_counts(sort=False).to_frame()


1325


,INTERVAL_DIAG_APPT
less_than_-3months,2
less_than_-2months,23
less_than_-1month,39
more_than_-1month,86
less_than_1month,87
more_than_1month,95
more_than_2months,97
more_than_3months,164
more_than_0.5yrs,198
more_than_1yr,350


In [51]:
HESOP_oligo.groupby('ATENTYPE')['PSEUDO_PATIENTID'].count()

ATENTYPE
1     211
2     842
3       1
4      15
5      47
7      20
8      47
10     12
11     69
12      1
13      6
21      8
22     43
25      2
28      1
Name: PSEUDO_PATIENTID, dtype: int64

In [54]:
HESOP_oligo.groupby('ATTENDED')['PSEUDO_PATIENTID'].count()

ATTENDED
2      68
3      64
4      82
5    1094
6      11
9       6
Name: PSEUDO_PATIENTID, dtype: int64

In [55]:
HESOP_oligo.groupby('FIRSTATT')['PSEUDO_PATIENTID'].count()

FIRSTATT
1     258
2    1011
3       8
4      46
X       2
Name: PSEUDO_PATIENTID, dtype: int64

In [56]:
HESOP_oligo.groupby('PRIORITY')['PSEUDO_PATIENTID'].count()

PRIORITY
1    1017
2     229
3      27
9      52
Name: PSEUDO_PATIENTID, dtype: int64

In [58]:
HESOP_oligo.groupby('REFSOURC')['PSEUDO_PATIENTID'].count()

REFSOURC
1      14
3     173
4      13
5     802
6      15
10      5
11    155
12      1
13     10
14     11
15      1
17      1
92      2
97    110
99     12
Name: PSEUDO_PATIENTID, dtype: int64

In [60]:
HESOP_oligo['WAITING'].replace(r'', np.nan, inplace = True)
HESOP_oligo.dropna(axis = 0, subset=['WAITING'], inplace = True)
bin = [0, 30, 60, 90, 180, 365, 1095, 1825, 10000]
# label = ['less_than_-3months', 'less_than_-2months', 'less_than_-1month', 'more_than_-1month', 'less_than_1month', 'more_than_1month', 'more_than_2months', 'more_than_3months', 'more_than_0.5yrs', 'more_than_1yr', 'more_than_3yrs', 'more_than_5yrs']
DDA_group = pd.cut(HESOP_oligo['WAITING'].astype(float).astype(int), bins = bin)

DDA_group.value_counts(sort=False).to_frame()

,WAITING
"(0, 30]",292
"(30, 60]",210
"(60, 90]",105
"(90, 180]",141
"(180, 365]",167
"(365, 1095]",152
"(1095, 1825]",28
"(1825, 10000]",4


In [61]:
HESOP_oligo.groupby('OPERSTAT')['PSEUDO_PATIENTID'].count()

OPERSTAT
1    352
8    640
9     98
      68
Name: PSEUDO_PATIENTID, dtype: int64

In [63]:
HESOP_oligo.groupby('MAINSPEF')['PSEUDO_PATIENTID'].count()

MAINSPEF
100      9
101      8
110      4
120      9
130     19
140      3
145      1
150    172
160      2
180      1
190      5
192      1
300     18
301      8
302      3
303      7
313      2
314      5
315     10
320      5
330      8
340      9
350      3
361      1
370     62
371      1
400     95
401     11
410      4
430      3
501      4
502     12
560      4
700      2
710      2
713      9
800    409
810     51
821      1
823      1
831      1
900      1
950     50
960    121
&        1
Name: PSEUDO_PATIENTID, dtype: int64

In [64]:
HESOP_oligo.groupby('TRETSPEF')['PSEUDO_PATIENTID'].count()

TRETSPEF
100      5
101      9
103      1
104      2
107      2
      ... 
722      1
800    479
812     75
840      3
&        2
Name: PSEUDO_PATIENTID, Length: 62, dtype: int64